# USGS dataretrieval Python Package NLDI Data Access Examples

This notebook provides examples of using the Python dataretrieval package to retrieve data from the United States  Geological Survey (USGS) Hydro Network-Linked Data Index (NLDI). The dataretrieval package provides a collection of functions to get data from the USGS Hydro Network-Linked Data Index (NLDI). For more information about NLDI visit the [USGS website](https://labs.waterdata.usgs.gov/docs/nldi/about-nldi/index.html) describing NLDI or [this blog post](https://waterdata.usgs.gov/blog/nldi-intro/) that covers basic features of the NLDI.

From the [NLDI API documentation](https://labs.waterdata.usgs.gov/api/nldi/swagger-ui/index.html): The NLDI is a search service that takes a watershed outlet identifier as a starting point, a navigation mode to perform, and the type of data desired in response to the request. It can provide geospatial representations of the navigation or linked data sources found along the navigation. It also has the ability to return landscape characteristics for the catchment the watershed outlet is contained in or the total upstream basin.

### Install the Package

Use the following code to install the package if it doesn't exist already within your Jupyter Python environment. Note the `nldi` option in the `dataretrieval` package installation. The default `dataretrieval` package installation does not support NLDI data access. You must run the dataretrieval install with the `nldi` option to ensure that you have the necessary capabilities. If you are running this notebook in the CUAHSI JuypyterHub server linked to HydroShare, you will want to run the following pip install command. The base `dataretrieval` package is already installed in the CUAHSI JupyterHub Python environment, but it does not include the NLDI option.

In [ ]:
!pip install dataretrieval[nldi]

Load the package so that you can use its functions in this notebook.

In [ ]:
from dataretrieval import nldi
from IPython.display import display
import folium

***

## Usage Examples

The dataretrieval package provides a number of functions to get data from the USGS NLDI. The following sections provide examples of how each of the available functions can be used

* `get_basin()`: function used to get the upstream basin boundary for any feature indexed by NLDI.
* `get_flowlines()`: function used to get upstream or downstream flowline data from NLDI.
* `get_features()`: function used to retrieve information about features from any source indexed by NLDI.
* `search()`: general function used to retrieve data from NLDI. Can be used in place of the functions listed above.

***

### Examples for the `get_basin()` function:

The following examples show how to use the `get_basin()` function from the dataretrieval package to retrieve the upstream basin boundary (the contributing watershed for a feature) from the USGS NLDI associated with any feature indexed by NLDI. 

The following arguments are supported:

* **feature_source** (string): The name of the NLDI feature source.
* **feature_id** (string): The identifier of the NLDI feature.
* **simplified** (boolean): If True, the data will be returned with simplified polygons. If False, the data will be returned as a single polygon (default is False).
* **split_catchment** (boolean): If True, the data will be returned with split catchment polygons. If False, the data will be returned as a single polygon (default is False) NOTE: Setting this to True may result in an error due to a known issue with NLDI API.
* **as_json** (boolean): If True, the data will be returned as a python dictionary. If False, the data will be returned as a geopandas dataframe (default is False).


#### Example 1: Get the upstream basin for a single feature from a single source.
In this example, we will retrieve the boundary for the upstream basin for a single monitoring site feature. For the monitoring site feature, we will use a USGS water quality monitoring station. The feature source is the Water Quality Portal (WQP), and the identifier of the feature is the USGS water quality monitoring site identifer prepended with the agency code for USGS (USGS-01031500). You can use the identifiers for any monitoring station indexed by the NLDI to retrieve its upstream watershed boundary.

In [ ]:
# set the parameters needed to retrieve data
feat_source = 'WQP'
feat_id = 'USGS-01031500'

Now we can call the `get_basin` function to get the coordinates of the polygon making up the basin boundary associated with this monitoring site - i.e., the watershed area upstream of the given monitoring site. The result will be returned as a geopandas dataframe unless the `as_json` argument is used and set to True.

In [ ]:
basin_gdf = nldi.get_basin(feature_source=feat_source, feature_id=feat_id)
display(basin_gdf)

If you want to get the basin boundary coordinate data in GeoJSON format, you can use the `as_json` argument in the function call (as_json=True)

In [ ]:
basin_json_data = nldi.get_basin(feature_source=feat_source, feature_id=feat_id, as_json=True)
print(basin_json_data)

Make a quick map of the selected monitoring station and the upstream boundary returned by `get_basin()`.

In [ ]:
# Get the feature associated with the monitoring site 
# More examples of how to use the get_features() function are given below
site_gdf = nldi.get_features(feature_source=feat_source, feature_id=feat_id)

# Set the Coordinate Reference System (CRS) for the GeoDataFrames 
# containing the basin boundary coordinates and the monitoring site
# epsg='4326' for WGS84
basin_gdf.set_crs(epsg='4326', inplace=True)
site_gdf.set_crs(epsg='4326', inplace=True)

# Create a base map using folium
m = folium.Map(location=[site_gdf.geometry.x[0], site_gdf.geometry.y[0]], zoom_start=10)

# Add the selected monitoring location and basin features to the map
folium.GeoJson(site_gdf, name='Monitoring Location').add_to(m)
folium.GeoJson(basin_gdf, name='Basin Boundary', color='red').add_to(m)

# Zoom the map to the bounds of the data
bounds = m.get_bounds()
m.fit_bounds(bounds)

# Add layer control to toggle layers
folium.LayerControl().add_to(m)

# Display the map
m

***

### Examples for the `get_flowlines()` function:

The following examples show how to use the `get_flowlines()` function from the dataretrieval package to get flowline data from the USGS NLDI. Flowlines can be traced upstream or downstream from any indexed feature (e.g., a USGS monitoring location), and the result can include mainstem only or tributaries. Flowlines are returned for the specified navigation in WGS84 latitude/longitude coordinats in GeoJSON format.

The following arguments are supported:

* **navigation_mode** (string): Navigation mode (allowed values are 'UM', 'DM', 'UT', 'DD'). 'UM' = Upstream on the mainstem. 'DM' = Downstream on the mainstem. 'UT' = Upstream with tributaries. 'DD' = Downstream with diversions.
* **feature_source** (string): The name of the NLDI feature source.
* **feature_id** (string): The identifier of the NLDI feature.
* **comid** (integer): COMID (required if feature_resource is not specified).
* **distance** (integer): Distance in kilometers (default is 5). This distance parameter dictates the distance to navigate.
* **as_json** (boolean): If True, the data will be returned as a python dictionary. If False, the data will be returned as a geopandas dataframe (default is False).

#### Example 1: Get the flowline data using feature_source and feature_id
Get the flowline data tracing upstream on the mainstem from a USGS water quality monitoring site with the result returned as a geopandas dataframe. In this example, we will trace upstream including tributaries and include the distance argument with a distance that is sufficiently large to retrieve all of the flowlines upstream of the monitoring station.

In [ ]:
feat_source = 'WQP'
feat_id = 'USGS-01031500'

flowlines_gdf = nldi.get_flowlines(navigation_mode='UT', feature_source=feat_source, feature_id=feat_id, distance=100)
display(flowlines_gdf)

Get the same flowline data with the result returned as GeoJSON (as_json=True)

In [ ]:
flowlines_json_data = nldi.get_flowlines(navigation_mode='UT', feature_source=feat_source, feature_id=feat_id, as_json=True)
print(flowlines_json_data)

Add the retrieved flowline data to the map for visualization of what is returned. To change the distance traced upstream on the flowlines, change the value of the distance argument in the `get_flowlines()` function call and set the distance upstream you want to trace. You can also change the navigation_mode argument to include or exclude tributaries.

In [ ]:
# Get the feature associated with the monitoring site 
# More examples of how to use the get_features() function are given below
site_gdf = nldi.get_features(feature_source=feat_source, feature_id=feat_id)

# Set the Coordinate Reference System (CRS) for the GeoDataFrames 
# containing the basin boundary coordinates and the monitoring site
# epsg='4326' for WGS84
site_gdf.set_crs(epsg='4326', inplace=True)
flowlines_gdf.set_crs(epsg='4326', inplace=True)

# Create a base map using folium
m = folium.Map(location=[site_gdf.geometry.x[0], site_gdf.geometry.y[0]], zoom_start=10)

# Add the selected monitoring location and basin features to the map
folium.GeoJson(site_gdf, name='Monitoring Location').add_to(m)
folium.GeoJson(basin_gdf, name='Basin Boundary', color='red').add_to(m)
folium.GeoJson(flowlines_gdf, name='Flowlines', color='blue').add_to(m)

# Zoom the map to the bounds of the data
bounds = m.get_bounds()
m.fit_bounds(bounds)

# Add layer control to toggle layers
folium.LayerControl().add_to(m)

# Display the map
m

#### Example 2: Get flowline data using a NHDPlus COMID
In some cases, you may wish to get flowline data associated with a NHDPlus COMID rather than flowlines associated with a monitoring station. The following shows how to get flowline data for a COMID as a geopandas dataframe.

In [ ]:
gdf = nldi.get_flowlines(navigation_mode='UM', comid=13294314)
display(gdf)

To get the same flowline data as GeoJSON (as_json=True) instead of as a geopandas dataframe, do the following.

In [ ]:
flowlines_json_data = nldi.get_flowlines(navigation_mode='UM', comid=13294314, as_json=True)
print(flowlines_json_data)

***

### Examples for the `get_features()` function:

The following examples show how to use the `get_features()` function from the dataretrieval package to get features data from the USGS NLDI. The `get_features()` function returns all features found along the specified navigation as points in WGS84 latitude/longitude endoced as GeoJSON.

The following arguments are supported:

* **data_source** (string): The name of the NLDI data source.
* **navigation_mode** (string): Navigation mode (allowed values are 'UM', 'DM', 'UT', 'DD'). 'UM' = Upstream on the mainstem. 'DM' = Downstream on the mainstem. 'UT' = Upstream with tributaries. 'DD' = Downstream with diversions.
* **feature_source** (string): The name of the NLDI feature source.
* **feature_id** (string): The identifier of the NLDI feature (required if feature_source is specified).
* **comid** (integer): COMID (required if feature_source is not specified).
* **distance** (integer): Distance in kilometers (default is 50). This distance parameter dictates the distance to navigate.
* **lat** (float): Latitude (required if feature for a specific location is specified).
* **long** (float): Longitude (required if feature for a specific location is specified).
* **as_json** (boolean): If True, the data will be returned as a python dictionary. If False, the data will be returned as a geopandas dataframe (default is False).

#### Example 1: Get all indexed features along a specified navigation path
Get all of the indexed features from a particular data source using a navigation path that traces upstream along the mainstem (UM) and uses as an origin for the trace a feature from a given feature source (e.g., a monitoring station from the Water Quality Portal). You can get any indexed features from any of the included data sources. Example data sources and the codes you need to retrieve features from those sources include:
* "census2020-nhdpv2" - 2020 Census Block - NHDPlusV2 Catchment Intersections
* "epa_nrsa" - EPA National Rivers and Streams Assessment
* "gfv11_pois" - USGS Geospatial Fabric V1.1 Points of Interest
* "huc12pp" - HUC12 Pour Points NHDPlusV2
* "npdes" - NPDES Facilities that Discharge to Water
* "nwisgw" - NWIS Groundwater Sites
* "nwissite" - NWIS Surface Water Sites
* "WQP" - Water Quality Portal
* "comid" - NHDPlus comid

There are a few others - for a complete list, consult the getDataSources endpoint of the NLDI API. For this example, we will trace upstream along the mainstem and find any NWIS surface water sites located along the mainstem of the river.

In [ ]:
feat_source = 'WQP'
feat_id = 'USGS-01031500'
features_gdf = nldi.get_features(data_source='nwissite', navigation_mode='UM', feature_source=feat_source, feature_id=feat_id)
display(features_gdf)

Add the returned features to the map.

In [ ]:
# Get the feature associated with the monitoring site 
# More examples of how to use the get_features() function are given below
site_gdf = nldi.get_features(feature_source=feat_source, feature_id=feat_id)

# Set the Coordinate Reference System (CRS) for the GeoDataFrames 
# containing the basin boundary coordinates and the monitoring site
# epsg='4326' for WGS84
site_gdf.set_crs(epsg='4326', inplace=True)
features_gdf.set_crs(epsg='4326', inplace=True)

# Create a base map using folium
m = folium.Map(location=[site_gdf.geometry.x[0], site_gdf.geometry.y[0]], zoom_start=10)

# Add the selected monitoring location and basin features to the map
folium.GeoJson(site_gdf, name='Monitoring Location').add_to(m)
folium.GeoJson(basin_gdf, name='Basin Boundary', color='red').add_to(m)
folium.GeoJson(flowlines_gdf, name='Flowlines', color='blue').add_to(m)
folium.GeoJson(features_gdf, name='Features', color='green').add_to(m)

# Zoom the map to the bounds of the data
bounds = m.get_bounds()
m.fit_bounds(bounds)

# Add layer control to toggle layers
folium.LayerControl().add_to(m)

# Display the map
m

Rather than using a water quality monitoring station as the orgin, you can also use a NHDPlus COMID as the origin for the trace. The code below does the same thing as the code above except it uses a COMID as the origin for the navigation. 

In [ ]:
gdf = nldi.get_features(data_source='nwissite', navigation_mode='UM', comid=13294314)
display(gdf)

#### Example 2: Get information about indexed features
Sometimes you may want to get information about indexed features without using any sort of navigation path. In this case, you can use `get_features()` without specifying a navigation_mode. The following code returns the feature information for a water quality monitoring station from the Water Quality Portal.

In [ ]:
gdf = nldi.get_features(feature_source='WQP', feature_id='USGS-01031500')
display(gdf)

#### Example 3: Get information about indexed features for a specific location
Get information about indexed features for a specific location by passing latitude and longitude coordinates into the `get_features()` function.

In [ ]:
gdf = nldi.get_features(lat=43.073051, long=-89.401230)
display(gdf)

***

### Examples for the `search()` function:

The following examples show how to use the `search()` function from the dataretrieval package to get data (basins, flowlines, and features) from the USGS NLDI. You can use the `search()` function instead of the `get_basin()`, `get_flowlines()`, and `get_features()` functions described above. The search function returns data as a python dictionary. 

The following arguments are supported:

* **feature_source** (string): The name of the NLDI feature source.
* **feature_id** (string): The identifier of the NLDI feature (required if feature_resource is specified).
* **navigation_mode** (string): Navigation mode (allowed values are 'UM', 'DM', 'UT', 'DD'). 'UM' = Upstream on the mainstem. 'DM' = Downstream on the mainstem. 'UT' = Upstream with tributaries. 'DD' = Downstream with diversions.
* **data_source** (string): The name of the NLDI data source.
* **find** (string): The specific data type to search for. Allowed values are 'basin', 'flowlines', and 'feature' (default is 'features').
* **comid** (integer): NHDPlus COMID (required if feature_source is not specified).
* **lat** (float): Latitude (required if feature for a specific location is specified).
* **long** (float): Longitude (required if feature for a specific location is specified).
* **distance** (integer): Distance in kilometers (default is 50). This distance parameter dictates the distance to navigate.

#### Example 1: Get the upstream basin for an indexed water quality monitoring station
Instead of using `get_basin()`, the `search()` function can be used to retrieve the upstream contributing area for a water quality station from the Water Quality Portal.

In [ ]:
# set the parameters needed to retrieve data
feat_source = 'WQP'
feat_id = 'USGS-01031500'

In [ ]:
basin_data = nldi.search(feature_source=feat_source, feature_id=feat_id, find="basin")
print(basin_data)

#### Example 2: Get flowlines data for a specified feature source
Instead of using `get_flowlines()`, the `search()` function can be used to retrieve the flowlines traced upstream or downstream from a feature. In this case, we can get the flowlines upstream along the mainstem for the water quality station in the last example.

In [ ]:
flowlines_data = nldi.search(navigation_mode='UM', feature_source=feat_source, feature_id=feat_id, find='flowlines')
print(flowlines_data)

#### Example 3: Get all features along a specified navigation path
Likewise, instead of using `get_features()`, the `search()` function can be used to retrieve all of the indexed features from a particular data source along a particular navigation path - in this case tracing upstream along the mainstem from the same water quality station in the previous examples.

In [ ]:
features_data = nldi.search(data_source='nwissite', navigation_mode='UM', feature_source=feat_source,
                            feature_id=feat_id, find='features')
print(features_data)